Unit Test 

In [ ]:
import unittest 
from Beacon_AutoTest import *
from pandas.util.testing import assert_frame_equal
import HtmlTestRunner

class TestCase_Beacon_AutoTest(unittest.TestCase):
    """ This test the first row of input """
    @classmethod
    def setUpClass(self):

        model_file = 'test_info_1.xlsx'   # model and its parameter
        data_file = 'RX_Data_Library.xlsx'  # data library and use case
        self.model_df= pd.read_excel(model_file, sheet_name='Model')
        self.usecase_df = pd.read_excel(data_file, sheet_name='TestUseCase')[["Row", "UseCase_set"]]     
        self.param_df = pd.read_excel(model_file, sheet_name='Default_ParametersSet')[["Row", "Sim_model","Parameters", "Modified Parameters", "UseCases"]]
        self.datalib_df = pd.read_excel(data_file, sheet_name='DataLibrary')        
        self.test_info_df= pd.read_excel('create_simulink_input_df.xlsx') # expected output for create_simulink_input
        self.entry= pd.read_excel("matlab_input_df.xlsx") # expected output for create_matlab_input
        
        self.input_rows_list= [0]
        self.test_df= self.param_df.iloc[self.input_rows_list]
        self.usecaselist= [self.test_df.iloc[0]["UseCases"]] 
             
        self.model_name= 'MP_test.slx'        
        self.error_flag= 0
        self.signal_id_with_lib_names= 'Lib3: (115,123,124,1108,1116,1117);Lib4: (179,1172)'                
        self.signal_no_list= [115, 123, 124, 1108, 1116, 1117, 179, 1172]        

    def test_input_row_numbers(self): # test whether list returned is a subset of list of possible row numbers
        possible_rows_list= range(0,len(self.param_df)) 
        self.assertTrue(set(input_row_numbers()).issubset(set(possible_rows_list)))
               
    def test_extract_param_range_str_to_dict(self):
        teststr= ((self.model_df[self.model_df["Sim_model"] == self.model_name]["Parameter_range_list"]).tolist())[0]
        inf= float('inf')
        testdict= extract_param_range_str_to_dict(self.model_name, self.model_df)
        # testing the keys in the dict
        subset1 = [key.split(':')[0] for key in testdict]
        subset2 = [item.split(':')[0] for item in teststr.split(',')]
        self.assertEqual(subset1, subset2)        
        # testing the values in the dict
        strset1 = [item.split(':')[1:] for item in (teststr.replace('(','').replace(')','').replace(' ','')).split(',')]
        subset1 =[]
        for ls in strset1:
            subset1 = subset1 + [[float(i) for i in ls]]
        subset2 = list(testdict.values())
        self.assertEqual(subset1, subset2)
        
    def test_param_verify(self):
        self.assertEqual(param_verify(self.error_flag, self.test_df, self.model_df), 0)
        
    def test_usecase_verify(self):
        self.assertEqual(usecase_verify(self.error_flag, self.test_df, self.usecase_df), 0)
        
    def test_reformat_usecase_list(self):
        self.assertEqual(reformat_usecase_list(self.signal_id_with_lib_names), ','.join(str(e) for e in self.signal_no_list))
    
    def test_convert_usecase_to_signal_list(self):
        self.assertEqual(convert_usecase_to_signal_list(self.usecase_df, self.usecaselist), self.signal_no_list)
                            
    def test_create_simulink_input_df(self):
        assert_frame_equal(create_simulink_input_df(self.test_df, self.usecase_df), self.test_info_df, check_dtype=False)
        
    def test_create_matlab_input_list(self):
        matlab_input= create_matlab_input_list(self.test_info_df, self.datalib_df, self.model_df)
        temp = list(np.full([24,9,9], np.nan))
        data = {'FileName':temp,
                  'Params': temp, 
                  'RunNum': temp, 
                  'Model': temp, 
                  'LogFolder': temp, 
                  'SignalPath': temp, 
                  'ModelPath': temp, 
                  'InitFunc': temp, 
                  'SignalNum': temp}
        matlab_input_df= pd.DataFrame(data)
        for row, ls in enumerate(matlab_input):
            for i, item in enumerate(ls):
                matlab_input_df.iloc[row,i]= item                
        assert_frame_equal(matlab_input_df, self.entry)
        
        
#unittest.main(argv=[''], verbosity=2, exit=False)     

if __name__ == "__main__":
    ## when you run this in Jupyter Notebook
    unittest.main(argv=['first-arg-is-ignored'], exit=False,testRunner=HtmlTestRunner.HTMLTestRunner(output='C:\\Users\\F53992\\Documents\\Bosheng_AutomationCodes\\GitCommit\\AutoTest\\AutoTestStepbyStep\\UnitTestOutput'))
 